In [1]:
import os
import json
from typing import Dict, Any
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import re
import json
import esprima
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

# Set your OpenAI API key
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get OpenAI API key from environment variables
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
TRANSACTIONS_CODE = """
/**
 * @description JavaScript client for EVM blockchain API operations
 * 
 * This module provides functions to interact with EVM blockchains through various APIs,
 * allowing token transfers, swaps, and price quotes using LiFi SDK and other services.
 */

/**
/**
 * Swap one token for another using LiFi
 * 
 * @param {Object} params - Swap parameters
 * @param {string} params.fromToken - Contract Address of input token
 * @param {string} params.toToken - Contract Address of output token
 * @param {string} params.fromAddress - User Wallet Address
 * @param {string|number} params.fromAmount - Amount to swap (in normal format)
 * @returns {Promise<Object>} Swap transaction data containing:
 *   - transactionRequest: Object containing:
 *     - value: Transaction value in hex format
 *     - to: Contract address to send transaction to
 *     - data: Transaction data in hex format
 *     - from: Sender address
 *     - chainId: Chain ID for the transaction
 *     - gasPrice: Gas price in hex format
 *     - gasLimit: Gas limit in hex format
 *   - estimate: Object with swap details including:
*       - tool: Tool used for the swap
*       - approvalAddress: Address requiring approval
*       - toAmountMin: Minimum output amount
*       - toAmount: Expected output amount
*       - fromAmount: Input amount
*       - feeCosts: Array of fee information
*       - gasCosts: Array of gas cost information
*       - executionDuration: Estimated execution time
*       - fromAmountUSD: Input amount in USD
*       - toAmountUSD: Output amount in USD
 */
async function swap({
  fromToken,
  toToken,
  fromAddress,
  fromAmount
})
"""

TRANSACTIONS_USAGE = """
// Example 1: Transfer POL
const {hash, caip2} = await sendTransaction({
  transactionRequest: {
    to: '0x1234567890123456789012345678901234567890', // Receiver Address
    value: weiToHex(toWei(0.1, 18)), // Amount Formatted to Hex
    chainId: 137 // Chain ID (constant)
  }
});

// Example 2: Swap Tokens
const swapQuote = await swap(
  "0x0000000000000000000000000000000000000000", // From Token
  "0x3c499c542cEF5E3811e1192ce70d8cC03d5c3359", // To Token
  wallet.address, // From Address
  "0.001" // From Amount
});
const transactionData = swapQuote.transactionRequest;
const { hash, caip2 } = await sendTransaction(transactionData);
"""

HELPER_FUNCTIONS = """
// Get Balances
/**
 * Get token balances for a wallet address on Polygon.
 *
 * @returns {Promise<Array<Object>>} Array of token balance objects, each with:
 *   - chain: 'polygon-mainnet'
 *   - balance: Human-readable balance (string)
 *   - denominatedBalance: Raw balance as string (string)
 *   - symbol: Token symbol (string)

 * @param {string} walletAddress - Wallet address to get balances for
 * @returns {Promise<Array>} Array of token balances with enriched metadata
 */
export async function getBalances(walletAddress) 

// Get Token Info
/**
 * Get token information from LiFi API.
 *
 * @param {string} token                Token address (e.g. "0x0000000000000000000000000000000000000000")
 * @returns {Promise<Object>}           Token information object as described below:
 *   - address:         Token contract address (string)
 *   - chainId:         Chain ID (number)
 *   - symbol:          Token symbol (string)
 *   - decimals:        Token decimals (number)
 *   - name:            Token name (string)
 *   - coinKey:         Coin key (string)
 *   - logoURI:         Logo URL (string)
 *   - priceUSD:        Price in USD (string)
 */
async function getTokenInfo(token)

// Get Token Market Data
/**
 * Fetch token market data from Mobula API by symbol or asset name.
 *
 * @param {string} symbol                Token symbol (e.g. "BTC") or asset name (e.g. "bitcoin")
 * @returns {Promise<Object>}           Token market data object as described above
 *   - id:                Unique token identifier (number)
 *   - name:              Token name (string)
 *   - symbol:            Token symbol (string)
 *   - decimals:          Default decimals (number)
 *   - logo:              Logo URL (string)
 *   - rank:              Market cap rank (number)
 *   - price:             Current price (number)
 *   - market_cap:        Market capitalization (number)
 *   - market_cap_diluted: Fully diluted market cap (number)
 *   - volume:            24h trading volume (number)
 *   - volume_change_24h: 24h volume change (number)
 *   - volume_7d:         7d trading volume (number)
 *   - liquidity:         Current liquidity (number)
 *   - liquidityMax:      Maximum liquidity (number)
 *   - ath:               All-time high price (number)
 *   - atl:               All-time low price (number)
 *   - off_chain_volume:  Off-chain trading volume (number)
 *   - is_listed:         Whether the token is listed (boolean)
 *   - price_change_1h:   Price change in the last hour (number, %)
 *   - price_change_24h:  Price change in the last 24h (number, %)
 *   - price_change_7d:   Price change in the last 7 days (number, %)
 *   - price_change_1m:   Price change in the last month (number, %)
 *   - price_change_1y:   Price change in the last year (number, %)
 *   - total_supply:      Total supply (number)
 *   - circulating_supply: Circulating supply (number)
 *   - contracts:         Array of contract objects for each supported blockchain, each with:
 *       - address:       Contract address (string)
 *       - blockchainId:  Chain ID (string or number)
 *       - blockchain:    Blockchain name (string)
 *       - decimals:      Token decimals on that chain (number)
 */
async function getTokenMarketData(symbol)
"""

BASELINE_JS = """
[CODE]
// Baseline function for EVM blockchain transactions
// This code provides the infrastructure for:
// 1. Creating wallets using Privy
// 2. Transaction signing with ethers.js
// 3. Transaction submission
// The AI model should focus on implementing the transaction creation logic. The TRANSACTIONS functions will be pre-defined.

/**
 * Main baseline function that orchestrates the entire process
 */
export async function baselineFunction(ownerAddress) {
  const wallet = await createWallet(ownerAddress);

  console.log("Address:", wallet.address);

  console.log(
    `Checking balance for wallet ${{wallet.address}} every 30 seconds until target is reached...`
  );

  // Keep checking until success - no timeout limit
  while (true) {
    try {
      const result = await checkBalance(wallet.address, 0.01); // Add appropiate POL value to checkBalance
      console.log("Result: ", result);

      if (result.success) {
        console.log("✅ Target balance achieved!");

        // ============================
        // ======= ENTER AI CODE =======

                
        // ======= END AI CODE  =======
        // ============================
      }

      console.log(
        "❌ Target not reached yet. Waiting 30 seconds before next check..."
      );
      // Wait 30 seconds before next check
      await new Promise((resolve) => setTimeout(resolve, 30000));
    } catch (error) {
      console.error("Error checking balance:", error.message);
      console.log("Retrying in 30 seconds...");
      // Wait 30 seconds before retrying
      await new Promise((resolve) => setTimeout(resolve, 30000));
    }
  }
}
[/CODE]
"""

In [3]:
CODER_PROMPT = """
    You are <Agent E1>, a trading agent launcher created by Xade for EVM blockchains.

    Your task is to generate code to run on a serverless function to execute a user's trading positions on EVM blockchains.
    
    You will be provided with a prompt containing all the information required to handle and execute the trading position.
    You will have access to all the functions you may need to include to achieve this task as well.

    KNOWLEDGE:
    - You will currently be working only on the Polygon Network. 
      - The Chain ID of the network is 137.
      - The symbol of the native token used to be MATIC, but it was renamed to POL. You will be using the symbol interchangeably.
      - Use POL when called the getTokenInfo function, and MATIC when calling the getTokenMarketData function.
      - The contract address of the native USDC is 0x3c499c542cEF5E3811e1192ce70d8cC03d5c3359. USDC.e is outdated, and is not to be used unless specified by the user.
    - The agent's wallet information is enclosed in the wallet variable.
      - wallet.id contains the wallet ID.
      - wallet.address contains the wallet address.

    RESOURCES:
      1. Transactions Documentation:
        {TRANSACTIONS_CODE}
        This snippet contains the docstring of functions to call the Transactions API to generate transaction data. 
        These functions will be pre-defined. You need to use them to generate transactions.
      2. Transactions Usage:
        {TRANSACTIONS_USAGE}
        This contains examples to actually execute the transactions onchain.
      3. Helper Functions:
        {HELPER_FUNCTIONS}
        This contains the docstring of helper functions such as getTokenInfo, getTokenMarketData, getBalances which can help execute trading strategies.
 
    RULES
      1. Do not access any external JavaScript libraries/packages. This may cause the script to fail.
      2. Use the Date() function when needed, nothing that needs to be installed.
      3. Always output the entire baselineFunction().

    EXECUTION WORKFLOW:
    
    When a user prompt arrives, follow this comprehensive approach:

    - Deeply understand the trading strategy requirements and identify all tokens/symbols that influence execution decisions
    - Map out price conditions, thresholds, trigger points, and what market conditions will trigger trades
    - Establish stop-loss, take-profit, position sizing logic, and timing constraints for execution windows
    - Map complex if-then scenarios for market-responsive execution and identify all market data needed
    - Catalog all available TRANSACTIONS CODE and HELPER FUNCTIONS, mapping which functions serve each strategy component
    - Design how market data flows through decision logic and plan optimal order of operations for execution
    - Anticipate failure points and recovery mechanisms while minimizing API calls and maximizing efficiency
    - Build execution logic with integrated scheduling using `setInterval()` for recurring strategies or `Date()` validation for specific execution windows
    - Implement dynamic decision-making based on real-time data and create functions that analyze multiple symbols simultaneously
    - Build sophisticated if-then-else logic for complex strategies and seamlessly integrate all transaction types
    - Insert all strategy logic within the designated AI code area with detailed console output for transparency
    - Implement try-catch blocks and failsafe mechanisms for robust error handling
    - Compare prices and trends across multiple tokens for cross-symbol analysis
    - Execute trades based on optimal market conditions and manage multiple positions with rebalancing strategies
    - Implement momentum-based trading decisions, contrarian strategies based on price deviations, and incorporate trading volume into decision-making
    - Use price action patterns for entry/exit signals and technical indicators
     
     BASELINE FUNCTION:
     {BASELINE_JS}

     ## OUTPUT FORMAT:
     Return a structured JSON object with the following key:
     - **code**: Complete baselineFunction() with integrated timing logic and strategy execution

     ## EXECUTION PRINCIPLES:
     - **Autonomous Operation**: Code must execute flawlessly without user intervention
     - **Timing Integration**: All scheduling logic must be built into the baseline function
     - **Market Data Utilization**: Leverage helper functions for comprehensive market analysis
     - **Strategic Sophistication**: Implement complex, multi-conditional trading strategies
     - **Error Resilience**: Include robust error handling and recovery mechanisms
     - **Performance Optimization**: Efficient code execution with minimal resource usage
     - **Transparency**: Comprehensive logging for strategy monitoring and debugging
     - **Risk Management**: Built-in safeguards for position sizing and loss mitigation
     
    """

In [4]:
SAMPLE_TRADING_PROMPT_1 = """
Buy 0.1 USDC using POL every 40 minutes if the price of BTC has fallen by more than 5% in the last 24 hours.
"""

SAMPLE_TRADING_PROMPT_2 = """
Create a sophisticated portfolio rebalancing strategy that maintains optimal asset allocation.

STRATEGY REQUIREMENTS:
- Maintain 40% USDC.e, 30% WETH, 20% WBTC, 10% POL allocation
- Rebalance when any asset deviates more than 5% from target allocation
- Execute rebalancing trades every 2 hours during weekdays
- Use mean reversion signals: buy when asset is 10% below 7-day average, sell when 10% above
- Monitor correlation between ETH and BTC - reduce exposure if correlation drops below 0.7
- Track total portfolio value and rebalance if it grows/shrinks by more than 15%
- Implement dynamic position sizing based on volatility (reduce size when volatility > 50%)
- Emergency liquidation if any single asset drops 20% in 4 hours

EXECUTION TIMING:
- Portfolio check every 30 minutes
- Rebalancing execution every 2 hours (10 AM, 12 PM, 2 PM, 4 PM, 6 PM UTC)
- Continuous monitoring for emergency conditions
- Skip weekends (Saturday-Sunday UTC)

ADVANCED FEATURES:
- Calculate portfolio Sharpe ratio and adjust strategy aggressiveness
- Implement volatility-adjusted position sizing
- Track and log all rebalancing actions with before/after allocations
- Include slippage protection (max 2% slippage tolerance)
- Implement gas fee optimization (batch trades when possible)

Please generate the complete portfolio management code with integrated timing and risk controls.
"""

SAMPLE_TRADING_PROMPT_3 = """
Create a market-making strategy that provides liquidity and captures spread profits.

STRATEGY REQUIREMENTS:
- Focus on USDC.e/POL pair trading
- Place buy orders 1% below current market price, sell orders 1% above
- Use 5% of available balance for each order
- Adjust spread based on volatility (wider spreads during high volatility)
- Cancel and replace orders every 10 minutes to stay competitive
- Execute arbitrage opportunities when price difference exceeds 0.5% across DEXs
- Monitor order book depth and adjust strategy accordingly
- Implement inventory management to avoid being too long/short in any asset

EXECUTION TIMING:
- Order placement and management every 5 minutes
- Market data refresh every 2 minutes
- Arbitrage scanning every 30 seconds
- Full strategy evaluation every hour

RISK MANAGEMENT:
- Maximum exposure: 20% of portfolio in any single direction
- Stop strategy if unrealized losses exceed 3%
- Implement position limits to prevent inventory buildup
- Monitor gas costs and pause strategy if fees exceed 0.1% of trade value

ADVANCED FEATURES:
- Dynamic spread adjustment based on market conditions
- Implement anti-MEV protection strategies
- Track and optimize for maximum capital efficiency
- Include profit/loss tracking and performance metrics

Please generate the complete market-making strategy code with integrated risk management.
"""


In [5]:
model = ChatOpenAI(model="o4-mini")

In [6]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", CODER_PROMPT),
    ("human", SAMPLE_TRADING_PROMPT_2)
  ])

prompt = prompt_template.format(
    TRANSACTIONS_CODE=TRANSACTIONS_CODE,
    TRANSACTIONS_USAGE=TRANSACTIONS_USAGE,
    BASELINE_JS=BASELINE_JS,
    HELPER_FUNCTIONS=HELPER_FUNCTIONS,
)

In [7]:
output = model.invoke(prompt)

In [8]:
print(output.content)

{"code":"export async function baselineFunction(ownerAddress) {\n  const wallet = await createWallet(ownerAddress);\n\n  console.log(\"Address:\", wallet.address);\n  console.log(`Checking balance for wallet ${wallet.address} every 30 seconds until target is reached...`);\n\n  // Keep checking until we have minimum POL balance\n  while (true) {\n    try {\n      const result = await checkBalance(wallet.address, 0.01); // 0.01 POL minimum to start\n      console.log(\"Result: \", result);\n\n      if (result.success) {\n        console.log(\"✅ Target balance achieved! Starting portfolio manager...\");\n\n        // ============================\n        // ======= ENTER AI CODE =======\n\n        // TARGET ALLOCATIONS\n        const TARGET_ALLOCATIONS = {\n          USDC: 0.4,\n          WETH: 0.3,\n          WBTC: 0.2,\n          POL: 0.1\n        };\n        const SYMBOLS = [\"USDC\", \"WETH\", \"WBTC\", \"POL\"];\n        const TOKEN_ADDRESSES = {\n          USDC: \"0x3c499c542cEF5E38

In [9]:
import json

def parse_model_output(output_content):
    """
    Parse the model output to extract JSON response.
    Handles various output formats including markdown code blocks.
    """
    try:
        # Remove markdown code block formatting if present
        content = output_content.strip()
        
        # Check if content is wrapped in markdown code blocks
        if content.startswith('```json'):
            # Extract content between ```json and ```
            start_idx = content.find('```json') + 7
            end_idx = content.rfind('```')
            if end_idx > start_idx:
                content = content[start_idx:end_idx].strip()
        elif content.startswith('```'):
            # Extract content between ``` and ```
            start_idx = content.find('```') + 3
            end_idx = content.rfind('```')
            if end_idx > start_idx:
                content = content[start_idx:end_idx].strip()
        
        # Parse JSON
        parsed_json = json.loads(content)
        return parsed_json
        
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        print(f"Raw content: {output_content}")
        return None
    except Exception as e:
        print(f"Unexpected error parsing output: {e}")
        print(f"Raw content: {output_content}")
        return None

def validate_code_output(parsed_output):
    """
    Validate that the parsed output contains the expected structure.
    """
    if not parsed_output:
        return False, "No output to validate"
    
    if not isinstance(parsed_output, dict):
        return False, "Output is not a dictionary"
    
    if 'code' not in parsed_output:
        return False, "Missing 'code' key in output"
    
    if not parsed_output['code']:
        return False, "Code field is empty"
    
    # Basic validation that it's JavaScript code
    code = parsed_output['code']
    if not code.strip().startswith('//') and not code.strip().startswith('export'):
        return False, "Code doesn't appear to be valid JavaScript"
    
    return True, "Output validation passed"

# Example usage:
output_content = output.content  # from your model
parsed = parse_model_output(output_content)
is_valid, message = validate_code_output(parsed)
print(is_valid, message)
if is_valid:
    print(parsed['code'])  # print first 500 chars of code

True Output validation passed
export async function baselineFunction(ownerAddress) {
  const wallet = await createWallet(ownerAddress);

  console.log("Address:", wallet.address);
  console.log(`Checking balance for wallet ${wallet.address} every 30 seconds until target is reached...`);

  // Keep checking until we have minimum POL balance
  while (true) {
    try {
      const result = await checkBalance(wallet.address, 0.01); // 0.01 POL minimum to start
      console.log("Result: ", result);

      if (result.success) {
        console.log("✅ Target balance achieved! Starting portfolio manager...");

        // ============================
        // ======= ENTER AI CODE =======

        // TARGET ALLOCATIONS
        const TARGET_ALLOCATIONS = {
          USDC: 0.4,
          WETH: 0.3,
          WBTC: 0.2,
          POL: 0.1
        };
        const SYMBOLS = ["USDC", "WETH", "WBTC", "POL"];
        const TOKEN_ADDRESSES = {
          USDC: "0x3c499c542cEF5E3811e1192ce70d8cC03d5c3